In [1]:
import torch
import random

import torch.nn.functional as F
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_mll
from botorch.utils import standardize
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.acquisition import UpperConfidenceBound
from botorch.optim import optimize_acqf
import torch.nn.functional as F
from botorch.acquisition import ExpectedImprovement



c:\Users\phili\Documents\workspace\botorch\botorch_contrast\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
BATCHSIZE = 20
TARGET = [0.34, 0.2]


#r = torch.tensor([[random.uniform(0, 1), random.uniform(0, 1)]], dtype=torch.double)  
def dif(tensor, target):
  target = torch.tensor([target], dtype=torch.double)
  r = target - tensor 
  r = torch.abs(r)
  r = torch.sum(r)
  #r = torch.clamp(r,0,1)
  return 1-torch.tensor([[r]], dtype=torch.double)
  


def getSample():
  return torch.tensor([[random.uniform(0, 1), random.uniform(0, 1)]], dtype=torch.double)  

X = torch.tensor([], dtype=torch.double)
Y = torch.tensor([], dtype=torch.double)
# Create a Standardize transform using the data

for i in range(BATCHSIZE):
  x = getSample()
  X = torch.cat((X, x), dim=0)
  y = dif(x, TARGET)
  Y = torch.cat((Y, y), dim=0)
  
  
Y =  F.normalize(Y, dim=None, p=2)

print(Y)


list(zip(X, Y))


tensor([[0.2802],
        [0.2165],
        [0.2412],
        [0.3131],
        [0.1290],
        [0.3682],
        [0.0393],
        [0.1645],
        [0.3353],
        [0.1727],
        [0.0148],
        [0.0795],
        [0.2196],
        [0.1514],
        [0.1332],
        [0.2359],
        [0.1091],
        [0.3306],
        [0.1344],
        [0.3239]], dtype=torch.float64)


[(tensor([0.6554, 0.0802], dtype=torch.float64),
  tensor([0.2802], dtype=torch.float64)),
 (tensor([0.7114, 0.0078], dtype=torch.float64),
  tensor([0.2165], dtype=torch.float64)),
 (tensor([0.4911, 0.5627], dtype=torch.float64),
  tensor([0.2412], dtype=torch.float64)),
 (tensor([0.0286, 0.2574], dtype=torch.float64),
  tensor([0.3131], dtype=torch.float64)),
 (tensor([0.1834, 0.7835], dtype=torch.float64),
  tensor([0.1290], dtype=torch.float64)),
 (tensor([0.2585, 0.3763], dtype=torch.float64),
  tensor([0.3682], dtype=torch.float64)),
 (tensor([0.9428, 0.5181], dtype=torch.float64),
  tensor([0.0393], dtype=torch.float64)),
 (tensor([0.0124, 0.5408], dtype=torch.float64),
  tensor([0.1645], dtype=torch.float64)),
 (tensor([0.5176, 0.3467], dtype=torch.float64),
  tensor([0.3353], dtype=torch.float64)),
 (tensor([0.4119, 0.7800], dtype=torch.float64),
  tensor([0.1727], dtype=torch.float64)),
 (tensor([0.6895, 0.8207], dtype=torch.float64),
  tensor([0.0148], dtype=torch.float64)),

In [5]:
train_Y = standardize(Y)
#train_Y = Y
gp = SingleTaskGP(X, train_Y)
mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
fit_gpytorch_mll(mll)
bounds = torch.stack([torch.zeros(2), torch.ones(2)])

In [6]:

# Define the Expected Improvement (EI) acquisition function
ei = ExpectedImprovement(gp, best_f=torch.min(Y))

# 'best_observed_value' is the best observed value from your optimization process so far
bounds = torch.stack([torch.zeros(2), torch.ones(2)])

# Optimize the EI acquisition function to find the next candidate point
candidate, acq_value = optimize_acqf(
    ei, bounds=bounds, q=1, num_restarts=20, raw_samples=150,
)

print(candidate)

tensor([[0.3080, 0.1756]])


In [ ]:

UCB = UpperConfidenceBound(gp, beta=0.1)
candidate, acq_value = optimize_acqf(
    UCB, bounds=bounds, q=1, num_restarts=20, raw_samples=150,
)
print(candidate)

In [ ]:
def optimize_qehvi(model, train_obj, sampler):
    """Optimizes the qEHVI acquisition function, and returns a new candidate and observation."""
    # partition non-dominated space into disjoint rectangles
    partitioning = NondominatedPartitioning(ref_point=ref_point, Y=train_obj)
    acq_func = qExpectedHypervolumeImprovement(
        model=model,
        ref_point=ref_point.tolist(),  # use known reference point
        partitioning=partitioning,
        sampler=sampler,
    )
    # optimize
    candidates, _ = optimize_acqf(
        acq_function=acq_func,
        bounds=problem_bounds,
        q=BATCH_SIZE,
        num_restarts=NUM_RESTARTS,
        raw_samples=RAW_SAMPLES,  # used for intialization heuristic
        options={"batch_limit": 5, "maxiter": 200, "nonnegative": True},
        sequential=True,
    )
    # observe new values
    new_x =  unnormalize(candidates.detach(), bounds=problem_bounds)
    return new_x